In [103]:
%run cpch_source.py

<Figure size 640x480 with 0 Axes>

# Generating CPCH p-values


This notebook provides a tutorial for using applying the cPCH test. 

The function which computes cPCH p-values, `cpch()`, has the following arguments:
- `T`: A M $\times$ m matrix of base test statistics. M represents the number of PCH's being simultaneously tested and m represents the number of base hypotheses per PCH test. Note, to test a single PCH, set `T` to be a dimension 1 $\times$ m array.
- `m`: The number of base hypotheses. `m` must be an integer $\geq 2$.
- `r`: The value that determines the PCH null to test (i.e. testing that less than r base hypotheses are null.) `r` must be an integer such that $2 \leq r \leq m$. 
- `f`: A combining function for the PCH test statistic, such as Fisher's, Bonferroni's, or Simes' (see `f_fisher` in `cpch_source.py` as an example)
- `pdf` (Default: `norm.pdf`): A function which outputs the pdf of a distribution evaluated at a given point (e.g. `norm.pdf` or `t.pdf` in `scipy.stats`).  
- `N` (Default: 10,000): The number of samples to use for the Monte Carlo procedure for computing cPCH p-values. This argument will only be used if m > 2 as we can compute cPCH p-values exactly when r=m=2.
- `dof`: This argument is only required when `pdf` = `t.pdf`. A M $\times$ m matrix of the degrees-of-freedoms corresponding element-wise to a M $\times$ m matrix of base test statistics. 

If `pdf` is not `norm.pdf` or `t.pdf`, then the user must specify a CDF function and truncated distribution sampling function via:
- `cdf`: A function for computing the cdf of a distribution evaluated at a given point. If `pdf` is `norm.pdf` or `t.pdf` (both from `scipy.stats`), then `cdf` is automatically set to `scipy.stats`'s `norm.cdf` or `t.cdf` respectively. If `pdf` is not `norm.pdf` or `t.pdf`, then the user must specify this argument herself.
- `trunc_rvs`: A function for computing random draws from a truncated distribution. If `pdf` is `norm.pdf` or `t.pdf` (both from `scipy.stats`), then `trunc_rvs` is automatically set to `truncnorm.rvs`(from `scipy.stats`) or `trunc_t`(define in `cpch_source.py`) respectively. If `pdf` is not `norm.pdf` or `t.pdf`, then the user must specify this argument herself. 

Users may specify a different one parameter location family as the distributional assumption on the base test statistics by providing their own custom functions for `pdf`, `cdf`, and `trunc_rvs` (see `trunc_t` in `cpch_source.py` for an example of a custom function for sampling from a truncated t-distribution). `trunc_t` takes in (float) values for the degrees-of-freedom (DOF) and location parameters and generates random draws from a truncated t-distribution with the provided DOF and location, with scale = 1.).

Additionally, users can provide `cpch()` with any combining function `f` of their choice, which takes in a M $\times$ m matrix matrix of base test statistics and an axis along which to compute the combining function along (see `f_fisher` in `cpch_source.py` as an example). Note, we define the cPCH test to reject when $f\left(\mathbf{T}_{(1:m-r+1)}\right)$ is large, so please define any custom `f` accordingly. 


## Single PCH Testing

We first simulate two $\mathcal{N}(2, 1)$ test statistics and test $H_0^{2/2}$. Note, we are in the alternative as both base test statistics have nonzero means. Since the Fisher's, Bonferroni's and Simes' tests are equivalent in this case (the Max-P test), the cPCH p-values for each combining functions are the same, as desired. By default, `cpch()` assumes normally distributed base test statistics.

In [104]:
np.random.seed(2211)

In [105]:
m = 2
r = 2
T = norm.rvs(loc = [2, 2], scale = 1).reshape(1, m)
#calculate cPCH p-values
#Fisher
c_fisher = cpch(T, m, r, f_fisher)
#Bonferroni
#c_bonferroni = cpch(T, m, r, f_bonferroni)
#Simes
c_simes = cpch(T, m, r, f_simes)
print('cPCH-Fisher: ' + str(c_fisher))
print('cPCH-Simes: ' + str(c_simes))
#print('cPCH-Bonferroni: ' + str(c_bonferroni))

cPCH-Fisher: [0.0065005]
cPCH-Simes: [0.0065005]


Use the `cpch_oracle()` function to produce a cPCH Oracle p-value. `cpch_oracle()` has the same arguments as `cpch()`, plus a new argument, `true_loc`, which is M $\times$ m numpy array of the true $\theta_i$ values for each of the base test statistics in `T`. When using  `cpch_oracle()`, the elements of `true_loc` **must** be in descending order across the rows (e.g., `true_loc = np.array([3, 2, 1, 0, 0]).reshape(1, 5)`) and the provided `T` must be sorted according to `true_loc` (e.g., `T = norm.rvs(loc = true_loc, scale = 1).reshape(1, 5)`).

In [106]:
# m = 2
# r = 2
# true_loc = np.array([2, 0]).reshape(1, m)
# T = norm.rvs(loc = true_loc, scale = 1).reshape(1, m)
# #calculate CPCH p-values
# #Fisher
# c_fisher = cpch_oracle(T, m, r, f_fisher, true_loc = true_loc)
# #Simes
# c_simes = cpch_oracle(T, m, r, f_simes, true_loc = true_loc)
# print('cPCH-Fisher: ' + str(c_fisher))
# print('cPCH-Simes: ' + str(c_simes))

In the above example, we do not provide a specific value for `N` since the cPCH p-value can be computed exactly when $r=m=2$. When $m > 2$, the user can configure `N`, the number of samples to use for the Monte Carlo procedure for computing cPCH p-values. The default value is $N = 10,000$. Below, we calculate the Fisher, Bonferroni, and Simes CPCH p-values for $m = 4, r = 3$ with $N = 100,000$.

In [107]:
m = 4
T = norm.rvs(loc = [0, 3, 3, 3], scale = 1).reshape(1, m)
r = 3
#calculate CPCH p-values
#Fisher
c_fisher = cpch(T, m, r, f_fisher, N = 100000)
#Simes
c_simes = cpch(T, m, r, f_simes, N = 100000)
print('cPCH-Fisher: ' + str(c_fisher))
print('cPCH-Simes: ' + str(c_simes))

cPCH-Fisher: [0.00800117]
cPCH-Simes: [0.00450639]


### With t-distributed base test statistics

When `pdf = t.pdf`, `cdf = t.cdf`, and `trunc_rvs = trunc_t`, `cpch()` will compute a cPCH p-values assuming the base test statistics are t-distributed. The `dof` argument enables the user to specify a degrees-of-freedom (DOF) for **each** of the M $\times$ m t-distributed base test statistic i.e., we do not require the base test statistics to have the same DOF.

In [108]:
m = 4
r = 3
#specify degrees of freedom for the test statistic of each of the m base hypotheses
dof = np.array([[10, 20, 30, 40]])
#specify theta for each test statistic
loc = np.array([0, 3, 3, 3])
T_t = t.rvs(dof, loc, scale = 1).reshape(1, m)
#calculate cPCH-fisher p-value
c_fisher_t = cpch(T_t, m, r, f_fisher, pdf = t.pdf, dof = dof)
print('cPCH-Fisher: ' + str(c_fisher_t))

cPCH-Fisher: [0.03964977]


## Multiple Testing

Below, we generate data according to the first multiple testing example in the Multiple PCH Testing section (Section 3.3.2) of our paper for $r=m=2$ and $M = 10000$. We set $\pi_1 = 0.3$, so on average, we expect (0.3)^2 = 0.09 of the total PCH's to be non-null. In other words, about 9% of the PCH's will have both base hypotheses being non-null.

In [109]:
m = 2
r = 2
p = 0.75
pi1 = 0.3
M = 10000
theta = np.array([2.5])

#generate matrix of indicators, 1 if base hypotheses is nonnull, 0 0/w
nonnull_ind = generate_indicator_matrix(pi1, p, theta, m, r, M)

mu_matrix = np.random.choice(theta, size = (M, m)) * nonnull_ind
#sample XX from normal with above true mu
X = norm.rvs(mu_matrix, scale = 1, size = (M, m))

In [110]:
cpch_pvals = cpch(X, m, r, f_fisher)

We can apply an FDR controlling procedure like Benjamini-Hochberg to the cPCH p-values and compute the power and FDR.

In [111]:
alpha = 0.1
cpch_decision = mt.multipletests(cpch_pvals, alpha, method = 'fdr_bh')[0]
#generate indicator variable of which PCH hypotheses are nonnull
pch_nonnull_ind = np.sum(nonnull_ind, axis = 1)== 2
#power
cpch_power = np.mean(cpch_decision[pch_nonnull_ind])
#fdr
cpch_fdr = sum(cpch_decision[np.logical_not(pch_nonnull_ind)])/max(1, sum(cpch_decision))
print('cPCH BH Power: ' + str(cpch_power))
print('cPCH BH FDR: ' + str(cpch_fdr))

cPCH BH Power: 0.1350943396226415
cPCH BH FDR: 0.05789473684210526


Comparing to the Max-P test with Benjamini-Hochberg, we see that the cPCH test has greater power while maintaining FDR control.

In [112]:
maxp_pvals = np.max(2 * (1-norm.cdf(np.abs(X))), axis = 1)
#apply Benjamini Hochberg to the p-values
maxp_decision = mt.multipletests(maxp_pvals, alpha, method = 'fdr_bh')[0]
#power
maxp_power = np.mean(maxp_decision[pch_nonnull_ind])
#fdr
maxp_fdr = sum(maxp_decision[np.logical_not(pch_nonnull_ind)])/max(1, sum(maxp_decision))
print('Max-P BH Power: ' + str(maxp_power))
print('Max-P BH FDR: ' + str(maxp_fdr))


Max-P BH Power: 0.1079245283018868
Max-P BH FDR: 0.0
